# Pose Detection and Matching with OpenPose




## OpenPose Installation
The cell below will handle setting up our environment and installing OpenPose.
Due to the "hash mismatch" error I encountered, I did a patchy solution and updated the values to the actual values received for each model from the output below, the I reran my build in the next cell.

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'

project_name = splitext(basename(git_repo_url))[0]

if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`
!mkdir openpose/images openpose/videos

Selecting previously unselected package liblmdb0:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../00-liblmdb0_0.9.24-1build2_amd64.deb ...
Unpacking liblmdb0:amd64 (0.9.24-1build2) ...
Selecting previously unselected package libgflags2.2.
Preparing to unpack .../01-libgflags2.2_2.2.2-2_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-2) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../02-libgflags-dev_2.2.2-2_amd64.deb ...
Unpacking libgflags-dev (2.2.2-2) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../03-libgoogle-glog0v5_0.5.0+really0.4.0-2_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.5.0+really0.4.0-2) ...
Selecting previously unselected package libunwind-dev:amd64.
Preparing to unpack .../04-libunwind-dev_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind-dev:amd64 (1.3.2-2build2.1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to un

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Here I rebuild with the correct "hash" values in CMakeLists.txt to the models. (in lines 985-994).

In [ ]:
#rebuilding openpose due to hash mismatch error
!cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`



-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - found
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found version "12.2") 
-- Building with CUDA.
-- CUDA detected: 12.2
-- Found cuDNN: ver. 8.9.6 found (include: /usr/include, library: /usr/lib/x

#Upload BODY25 model manually
Due to issues I faced while building my environment  [(an issue faced by other OpenPose users as well)](https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/1602#issuecomment-641653411) I manually uploaded the model configurations to the /openpose/models folder, before moving on to run the demo. The link to download the models (for all including BODY25, COCO, and MPI are [here](https://drive.google.com/file/d/1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh).

## Run OpenPose Demo on Example Videos to Ensure Environment is Working
Now, I will first run the code on their example to ensure the installation was done properly before running on the infant videos.

In [ ]:
#!rm openpose.avi
#!rm -r ./openpose/output/
#!rm output.mp4
! cd openpose && ./build/examples/openpose/openpose.bin --model_pose COCO --video examples/media/video.avi --write_json ./output/ --display 0  --write_video ../openpose.avi --keypoint_scale 3
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

##Visualize Demo Result

In [ ]:
#the code to generate this mp4 video inline was used with help by ChatGPT
def show_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_mp4_video('output.mp4', width=960, height=720)

#Let's Move to Processing them on our Infant Videos

Using our processed videos that I manually uploaded using the URL and scripts located in the repo, let's run the OpenPose algorithm on them and see how they look!

For the cell below, I created a bash script that runs the same command as above to run OpenPose on all three classes (sitting, sleeping, standing located in the /openpose/videos/ directory) and save the output video (original video overlayed with detected poses) and the keypoints in the /openpose/videos/output directory for each subclass.


In [5]:
!pwd #should be in /content
!chmod 755 ./openpose/process_videos.sh #making sure the bash script is given right permissions to run
!./openpose/process_videos.sh

/content
Starting OpenPose demo...
Configuring OpenPose...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enab

##Save output to local disk
Now we will save the output files that include the keypoints detected in all the videos as well as the videos overlayed with the keypoints.


In [6]:
!zip -r openpose_output.zip /content/openpose/videos/output
from google.colab import files
files.download('/content/openpose_output.zip')

  adding: content/openpose/videos/output/ (stored 0%)
  adding: content/openpose/videos/output/sleeping/ (stored 0%)
  adding: content/openpose/videos/output/sleeping/keypoints_sleeping_1.json/ (stored 0%)
  adding: content/openpose/videos/output/sleeping/keypoints_sleeping_1.json/sleeping_1_000000000082_keypoints.json (deflated 56%)
  adding: content/openpose/videos/output/sleeping/keypoints_sleeping_1.json/sleeping_1_000000000230_keypoints.json (deflated 55%)
  adding: content/openpose/videos/output/sleeping/keypoints_sleeping_1.json/sleeping_1_000000000001_keypoints.json (deflated 56%)
  adding: content/openpose/videos/output/sleeping/keypoints_sleeping_1.json/sleeping_1_000000000189_keypoints.json (deflated 55%)
  adding: content/openpose/videos/output/sleeping/keypoints_sleeping_1.json/sleeping_1_000000000236_keypoints.json (deflated 55%)
  adding: content/openpose/videos/output/sleeping/keypoints_sleeping_1.json/sleeping_1_000000000044_keypoints.json (deflated 55%)
  adding: cont

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Processing Output Data
- in our case our output is already normalized between 0 and 1 using the --keypoint_scale flag we used before runnning out video on OpenPose
-We will use a prertained ST-GCN  model. The dataset the model was trained on was the Kinetics dataset where the estimated joint locations of 18  joints are represented as tuples of (X, Y, C).
- The training data used for the model uses the 2D coordinates (X, Y) in the pixel coordinate system and confidence scores C for the 18 human joints detected from Openpose.
- Our poses (above) above were extracted using the BODY25 model which is comprised of [25 body parts consisting of COCO + foot](https://github.com/lncarter/Openpose/blob/master/doc/output.md#reading-saved-results). We will shave off the last 7 joints for the foot keypoints for the purpose of keeping our data similar to the data used to  train ST-GCN.





##Understanding the structure of data used to evaluate ST-GCN
Here I am unpacking the data of one json file from an action video to understand how to format my data for the ST-GCN

In [38]:
#sample evaluation data json file from ST-GCN model
kinetics = json.load(open('/content/openpose/__lt03EF4ao.json'))
print(kinetics.keys()) #'data', 'label', 'label_index'
print(len(kinetics['data'])) #a list of all the frames and each frame is a dictionary (length should be number of frames)
print(kinetics['data'][0].keys()) #a dictionary for that frame with keys 'frame_index' (number), 'skeleton'
print(len(kinetics['data'][0]['skeleton'])) #a skeleton is a list of the pose coordinates of all humans detected in the frame
print(kinetics['data'][0]['skeleton'][0]) #a dictionary for each pose coordinate where 'pose' is a list of all coordinates and 'score' is the associated confidence
print(len(kinetics['data'][0]['skeleton'][0]['pose'])) #length is 36 where ther are 18 keypoints each with 2 coordinates
print(len(kinetics['data'][0]['skeleton'][0]['score'])) #18 scores, 1 for each joint

dict_keys(['data', 'label', 'label_index'])
249
dict_keys(['frame_index', 'skeleton'])
2
{'pose': [0.612, 0.639, 0.624, 0.663, 0.598, 0.663, 0.585, 0.715, 0.585, 0.666, 0.646, 0.663, 0.651, 0.723, 0.628, 0.739, 0.606, 0.75, 0.589, 0.818, 0.597, 0.889, 0.63, 0.753, 0.626, 0.818, 0.618, 0.891, 0.61, 0.636, 0.614, 0.636, 0.0, 0.0, 0.628, 0.633], 'score': [0.809, 0.858, 0.849, 0.757, 0.655, 0.868, 0.734, 0.717, 0.812, 0.859, 0.845, 0.769, 0.827, 0.675, 0.681, 0.763, 0.0, 0.693]}
36
18


##Function to process and combine all JSON files for one video
Since our output from openpose for each video is a series of 300 JSON siles (one for each frame in the video) and its corresponding poses in each, the function below takes in the file path to the folder containing all frame pose data for a single video, and returns a dictionary formatted to identically to the input for evaluation done in teh ST-GCN paper as described [here](https://arxiv.org/pdf/1801.07455.pdf)


In [81]:
def process_video(video_keypoints_path, action, label_index):
  video_frames = [pos_json for pos_json in os.listdir(video_keypoints_path) if pos_json.endswith('.json')]

  video_data = {"data":[], "label":action, "label_index":label_index} #append all frame_data to data
  frame_data = {"frame_index":0, "skeleton":[]}

  #iterate through each frame for each video
  #each keypoint is stored consecutviely (X, Y, C)  as for a total of 25 keypoints
  frame_index = 0
  for frame in video_frames:
    pose_dict = json.load(open(video_keypoints_path+frame))
    people_detected =  pose_dict['people']
    skeleton = []

    #append all skeletons deteected
    for person in people_detected:
      keypoints = person['pose_keypoints_2d'][:54] #we only want 18 COCO keypoints not foot so lets splice the rest
      partitioned_keypoints = [keypoints[i:i+3] for i in range(0, len(keypoints), 3)] #list of lists (X, Y, C) for each keypoint

      pose = []
      score = []
      for keypoint in partitioned_keypoints:
        pose += keypoint[0:2]
        score.append(keypoint[2])

      skeleton.append({'pose': pose, 'score':score})


    frame_data = {"frame_index": frame_index, 'skeleton':skeleton}
    video_data['data'].append(frame_data)

    frame_index += 1

  return video_data


In [85]:
import pandas as pd
import json
#from google.colab import drive
#drive.mount('/content/drive')


actions = ['sitting', 'standing', 'sleeping']
output_folder = '/content/openpose/videos/output/' #get all outputs for each video in sitting class

label_index = 0

#iterate through each class
for action in actions:
  #iterate through all 4 videos in each class
  for video_num in range(4):
    video_keypoints_path = '/content/openpose/videos/output/'+ action+ '/keypoints_'+action+'_'+str(video_num)+'.json/'  # Update this with the actual path
    video_data = process_video(video_keypoints_path, action, label_index)

    #File path to save the JSON file
    file_path = '/content/openpose/videos/json_processed/'+action+'/'+action+'_'+str(video_num)+'_processed.json'

    #Write dictionary to JSON file
    with open(file_path, 'w') as json_file:
        json.dump(video_data, json_file)

  label_index += 1

##Save and download all processed JSON files locally

In [87]:
!zip -r openpose_json_processed.zip /content/openpose/videos/json_processed
from google.colab import files
files.download('/content/openpose_json_processed.zip')

  adding: content/openpose/videos/json_processed/ (stored 0%)
  adding: content/openpose/videos/json_processed/sleeping/ (stored 0%)
  adding: content/openpose/videos/json_processed/sleeping/sleeping_2_processed.json (deflated 78%)
  adding: content/openpose/videos/json_processed/sleeping/sleeping_1_processed.json (deflated 67%)
  adding: content/openpose/videos/json_processed/sleeping/sleeping_0_processed.json (deflated 68%)
  adding: content/openpose/videos/json_processed/sleeping/sleeping_3_processed.json (deflated 74%)
  adding: content/openpose/videos/json_processed/standing/ (stored 0%)
  adding: content/openpose/videos/json_processed/standing/standing_1_processed.json (deflated 67%)
  adding: content/openpose/videos/json_processed/standing/standing_3_processed.json (deflated 68%)
  adding: content/openpose/videos/json_processed/standing/standing_0_processed.json (deflated 67%)
  adding: content/openpose/videos/json_processed/standing/standing_2_processed.json (deflated 66%)
  ad

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>